In [1]:
# Instalación de la librería para realizar acciones en la base de datos MySQL
#pip install mysql-connector-python

In [18]:
# Se importa la librería para trabajar con MySQL
import mysql.connector

In [3]:
# Función para solcitar el número de estudiantes que tiene un curso
def obtener_numero_estudiantes():
    while True:
        try:
            numero_estudiantes = int(input("Ingrese el número de estudiantes: "))
            if numero_estudiantes <= 0:
                print("Ingrese un número válido (mayor que 0).")
            else:
                return numero_estudiantes
        except ValueError:
            print("Ingrese un número válido.")

In [19]:
# Función para solicitar el id_estudiante (identificador del estudiante)
def obtener_id_estudiante():
    while True:
        try:
            id_estudiante = int(input("Ingrese el ID numérico del estudiante: "))
            if id_estudiante <= 0:
                print("Ingrese un ID válido (mayor que 0).")
            else:
                return id_estudiante
        except ValueError:
            print("Ingrese un ID válido.")

In [5]:
# Función para solicitar el id_materia (identificador de la materia)
def obtener_id_materia():
    while True:
        try:
            id_materia = int(input("Ingrese el ID de la materia: "))
            if id_materia <= 0:
                print("Ingrese un ID válido (mayor que 0).")
            else:
                return id_materia
        except ValueError:
            print("Ingrese un ID válido.")

In [6]:
# Función para almacenar las calificaciones de los estudiantes
def ingresar_calificaciones(numero_calificaciones):
    # Se obtiene el ID de la materia mediante la función obtener_id_materia()
    id_materia = obtener_id_materia()
    
    # Se obtiene el número de estudiantes mediante la función obtener_numero_estudiantes()
    numero_estudiantes = obtener_numero_estudiantes()
    
    # Se crea un diccionario vacío para almacenar las calificaciones de cada estudiante
    calificaciones_por_estudiante = {}
    
    # Se inicia un bucle que iterará por cada estudiante
    for i in range(1, numero_estudiantes + 1):
        
        # Se obtiene el ID del estudiante mediante la función obtener_id_estudiante()
        id_estudiante = obtener_id_estudiante()
        
        # Se crea una lista vacía para almacenar las calificaciones del estudiante actual
        calificaciones = []
        
         # Se inicia un bucle que iterará por cada calificación que se debe ingresar para el estudiante actual
        for j in range(1, numero_calificaciones + 1):
            while True:
                try:
                    # Se solicita al usuario ingresar la calificación para el estudiante y la materia actual
                    calificacion = float(input(f"Ingrese la calificación {j} del estudiante {i}: "))
                    if calificacion < 0 or calificacion > 100:
                        print("Ingrese una calificación válida (entre 0 y 5).")
                    else:
                        # Si la calificación es válida, se agrega a la lista de calificaciones del estudiante
                        calificaciones.append(calificacion)
                        break
                except ValueError:
                    print("Ingrese una calificación válida.")
        # Se agrega al diccionario calificaciones_por_estudiante el par (clave-valor) 
        # que representa al estudiante y sus calificaciones
        calificaciones_por_estudiante[(id_estudiante, id_materia)] = calificaciones
    return calificaciones_por_estudiante

In [7]:
def guardar_en_mysql(calificaciones_por_estudiante):
    # Establecer la conexión con la base de datos MySQL
    conexion = mysql.connector.connect(
        host="192.168.0.101",
        user="docente1",
        password="pwdocente1",
        database="notas"
    )

    # Crear una tabla llamada 'calificaciones' si no existe
    cursor = conexion.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS calificaciones (id_registro INT AUTO_INCREMENT PRIMARY KEY, id_estudiante INT, id_materia INT, calificacion FLOAT)")

    # Insertar las calificaciones en la tabla 'calificaciones'
    for (id_estudiante, id_materia), calificaciones in calificaciones_por_estudiante.items():
        for calificacion in calificaciones:
            cursor.execute("INSERT INTO calificaciones (id_estudiante, id_materia, calificacion) VALUES (%s, %s, %s)", (id_estudiante, id_materia, calificacion))

    # Guardar los cambios y cerrar la conexión
    conexion.commit()
    cursor.close()
    conexion.close()
    print("Datos guardados en la base de datos")

In [8]:
# Fución para calcular el promedio de notas de cada estudiante
def calcular_promedio_por_estudiante(calificaciones_por_estudiante):
    # Se crea un diccionario vacío para almacenar los promedios de calificaciones por estudiante
    promedios = {}
    # Se inicia un bucle que iterará por cada estudiante y sus calificaciones en el diccionario calificaciones_por_estudiante
    for estudiante, calificaciones in calificaciones_por_estudiante.items():
        # Se calcula el promedio de las calificaciones para el estudiante actual
        promedio = sum(calificaciones) / len(calificaciones)
        # Se agrega al diccionario promedios el par (clave-valor) que representa al estudiante y su promedio de calificaciones
        promedios[estudiante] = promedio
    return promedios

In [9]:
# Función que calcula la media de las notas de todo el curso
def calcular_promedio(calificaciones_por_estudiante):
    total_calificaciones = 0
    sumatoria = 0
    
    # Iteramos por cada estudiante y sus calificaciones en el diccionario 'calificaciones_por_estudiante'
    for calificaciones in calificaciones_por_estudiante.values():
        # Incrementamos 'total_calificaciones' con la cantidad de calificaciones del estudiante actual
        total_calificaciones += len(calificaciones)
        
        # Sumamos todas las calificaciones del estudiante actual y lo agregamos a 'sumatoria'
        sumatoria += sum(calificaciones)
    
    # Calculamos el promedio dividiendo la 'sumatoria' entre el 'total_calificaciones'
    promedio = sumatoria / total_calificaciones
    return promedio

In [16]:
def main():
    # Se solicita al usuario que ingrese el número de calificaciones por estudiante
    numero_calificaciones = int(input("Ingrese el número de calificaciones por estudiante: "))
    
    # Se llama a la función 'ingresar_calificaciones(numero_calificaciones)' 
    # para obtener las calificaciones ingresadas por el usuario
    calificaciones_por_estudiante = ingresar_calificaciones(numero_calificaciones)
    
    # Se llama a la función 'guardar_en_mysql(calificaciones_por_estudiante)' 
    # para almacenar las calificaciones en la base de datos MySQL
    guardar_en_mysql(calificaciones_por_estudiante)
    
    # Se llama a la función 'calcular_promedio_por_estudiante(calificaciones_por_estudiante)' 
    # para obtener los promedios de calificaciones por estudiante
    promedios_por_estudiante = calcular_promedio_por_estudiante(calificaciones_por_estudiante)
    
    # Se imprime el id del estudiante, con el id de la materia y el respectivo promedio final de notas
    print("\nPromedio de calificaciones por estudiante:\n")
    for estudiante, promedio in promedios_por_estudiante.items():
        print(f"Estudiante id: {estudiante[0]}, en la Materia id: {estudiante[1]} obtuvo una nota final de: {promedio:.2f}")
    
    # Muestra el promedio de todo el curso
    promedio_curso = calcular_promedio(calificaciones_por_estudiante)
    print(f"\nEl promedio de calificaciones del curso es de: {promedio_curso:.2f}")

In [17]:
# Ejecuta la función main
main()

Ingrese el número de calificaciones por estudiante: 4
Ingrese el ID de la materia: 10001
Ingrese el número de estudiantes: 3
Ingrese el ID numérico del estudiante: 101010
Ingrese la calificación 1 del estudiante 1: 4
Ingrese la calificación 2 del estudiante 1: 4.5
Ingrese la calificación 3 del estudiante 1: 5
Ingrese la calificación 4 del estudiante 1: 4.5
Ingrese el ID numérico del estudiante: 202020
Ingrese la calificación 1 del estudiante 2: 2
Ingrese la calificación 2 del estudiante 2: 3
Ingrese la calificación 3 del estudiante 2: 4
Ingrese la calificación 4 del estudiante 2: 3
Ingrese el ID numérico del estudiante: 303030
Ingrese la calificación 1 del estudiante 3: 3
Ingrese la calificación 2 del estudiante 3: 3.5
Ingrese la calificación 3 del estudiante 3: 4
Ingrese la calificación 4 del estudiante 3: 3.5
Datos guardados en la base de datos

Promedio de calificaciones por estudiante:

Estudiante id: 101010, en la Materia id: 10001 obtuvo una nota final de: 4.50
Estudiante id: 202